In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv')
df.drop(columns=['id','Unnamed: 32'], inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

In [ ]:
X_train_tensor=torch.from_numpy(X_train).float()
X_test_tensor=torch.from_numpy(X_test).float()
Y_train_tensor=torch.from_numpy(Y_train).float()
Y_test_tensor=torch.from_numpy(Y_test).float()

In [ ]:
class MySimpleNNModel(nn.Module):
    def __init__(self, num_features):
      super().__init__()
      self.linear = nn.Linear(num_features, 1)
      self.sigmoid = nn.Sigmoid()

    def forward(self,features):
      out = self.linear(features)
      out = self.sigmoid(out)
      return out

In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
# define loss function
loss_function = nn.BCELoss()

In [ ]:
# Now we need to train the model
# a) Create the model instance
model=MySimpleNNModel(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  # b) Forward pass
  y_pred=model(X_train_tensor)

  # c) loss calculation
  loss = loss_function(y_pred, Y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # d) backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7115992307662964
Epoch: 2, Loss: 0.5373331308364868
Epoch: 3, Loss: 0.44587069749832153
Epoch: 4, Loss: 0.3892281949520111
Epoch: 5, Loss: 0.3500586152076721
Epoch: 6, Loss: 0.32099372148513794
Epoch: 7, Loss: 0.29835814237594604
Epoch: 8, Loss: 0.2800992727279663
Epoch: 9, Loss: 0.2649734616279602
Epoch: 10, Loss: 0.25217923521995544
Epoch: 11, Loss: 0.24117501080036163
Epoch: 12, Loss: 0.2315802425146103
Epoch: 13, Loss: 0.22311894595623016
Epoch: 14, Loss: 0.2155855894088745
Epoch: 15, Loss: 0.2088235318660736
Epoch: 16, Loss: 0.20271115005016327
Epoch: 17, Loss: 0.1971522569656372
Epoch: 18, Loss: 0.1920696198940277
Epoch: 19, Loss: 0.18740054965019226
Epoch: 20, Loss: 0.18309332430362701
Epoch: 21, Loss: 0.1791049838066101
Epoch: 22, Loss: 0.17539936304092407
Epoch: 23, Loss: 0.17194584012031555
Epoch: 24, Loss: 0.16871820390224457
Epoch: 25, Loss: 0.1656939536333084


In [ ]:
with torch.no_grad():
  y_pred=model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == Y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5215451121330261
